<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamA/blob/main/4_create_factsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitungen

## Import der notwendigen Packete und bereitstellen der Dateien in Google Drive

In [ ]:
# !pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import pandas as pd
import numpy as np
import sqlite3
from scipy import stats
# import pandas_profiling
# from datetime import datetime
# import re
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# from pandas_profiling import ProfileReport
# will make plot outputs appear and stored within the notebook.
%matplotlib inline
from sqlite3 import Error
import folium
from folium import plugins
from folium.plugins import HeatMap
import json
import requests

from google.colab import drive
drive.mount('/content/drive')

# Path
material_path = "/content/drive/MyDrive/M3_DM_GrpA"

# path of datawarehouse
DB_DWH_PATH = material_path + f"/DWH_MAPCOV.db"

dwh_db = sqlite3.connect(DB_DWH_PATH)
dwh_cursor = dwh_db.cursor()

## Suche nach dem Beginn der Covid-19 Erkrankung pro Patient und dieser Filter wird in anderen Tabellen verwendet

Conditions: Filter nach Conditions, die parallel zu COVID aufgetreten sind.  
Erster Part enthält den allgemeien Filter in der Conditions Tabelle nach Start und Stop von COVID-19.  
Danach Merge mit der Conditions Tabelle OHNE 'COVID-19'. Das Ergebnis ist, dass hinter jedem Condition das Start/Stop Date von COVID steht. Nun kann eine Abfrage (Query) geschrieben werden, die die Datumsangaben vergleicht und filtert.

In [ ]:
df_conditions_filter = pd.read_sql_query("SELECT START AS stdt_covid, STOP AS endt_covid, PATIENT FROM conditions_INFO WHERE DESCRIPTION = 'COVID-19'", dwh_db)
print("Patients with Covid:", df_conditions_filter.shape)
df_cond_other = pd.read_sql_query("SELECT START, STOP, PATIENT, DESCRIPTION FROM conditions_info WHERE DESCRIPTION NOT LIKE '%COVID%'", dwh_db)
print("Number of other conitons than Covid:",df_cond_other.shape)
df_cond_covid = pd.merge(df_cond_other, df_conditions_filter, left_on="PATIENT", right_on="PATIENT",how="inner")
print("Merge:",df_cond_covid.shape)
df_cond_covid_qry = df_cond_covid.query('(STOP >= stdt_covid or STOP == "") and (START <= endt_covid or endt_covid == "") and DESCRIPTION.str.contains("(disorder)")')
print("Filter for cond within the same time as COVID:",df_cond_covid_qry.shape)
# df_cond_covid_qry.head()
df_cond_grp = df_cond_covid_qry.groupby("PATIENT")["DESCRIPTION"].count().rename("Disorder")
#print(df_cond_grp)

# Alternativ: Lists the unique vlaues of the column 'DESCRIPTION'
# df_cond_covid_qry.DESCRIPTION.unique()

Abfrage in devices_info nach zeitgleichem Auftreten mit COVID

In [ ]:
df_dev_all = pd.read_sql_query("SELECT Patient, START, STOP, DESCRIPTION FROM devices_INFO WHERE DESCRIPTION = 'Mechanical ventilator (physical object)'", dwh_db)
df_dev_covid = pd.merge(df_dev_all, df_conditions_filter, left_on="PATIENT", right_on="PATIENT",how="inner")
print("Merge:",df_dev_covid.shape)
df_dev_covid_qry = df_dev_covid.query('(STOP >= stdt_covid or STOP == "") and (START <= endt_covid or endt_covid == "")')
print("Query:",df_dev_covid_qry.shape)
# print(df_dev_covid_qry)
df_dev_grp = df_dev_covid_qry.groupby("PATIENT")["DESCRIPTION"].count().rename("Ventilation")
#print(df_dev_grp)

Abfrage in medications_info nach zeitgleichem Auftreten mit COVID

In [ ]:
df_med_all = pd.read_sql_query("SELECT Patient, START, STOP, DESCRIPTION, REASONDESCRIPTION FROM medications_info WHERE REASONDESCRIPTION = 'COVID-19'", dwh_db)
df_med_covid = pd.merge(df_med_all, df_conditions_filter, left_on="PATIENT", right_on="PATIENT",how="inner")
print("Merge:",df_med_covid.shape)
df_med_covid_qry = df_med_covid.query('(STOP >= stdt_covid or STOP == "") and (START <= endt_covid or endt_covid == "")')
print("Query:",df_med_covid_qry.shape)
#print(df_med_covid_qry)
df_med_grp = df_med_covid_qry.groupby("PATIENT")["DESCRIPTION"].count().rename("Medications")
#print(df_med_grp)
#df_med_covid_qry = df_med_covid_qry.drop(['stdt_covid', 'endt_covid', 'REASONDESCRIPTION'], axis=1)

Abfrage in observations_info

In [ ]:
df_obs_all = pd.read_sql_query("SELECT PATIENT, DATE, DESCRIPTION, VALUE, UNITS, TYPE FROM observations_info", dwh_db)
df_obs_covid = pd.merge(df_obs_all, df_conditions_filter, left_on="PATIENT", right_on="PATIENT",how="inner")
print("Merge:",df_obs_covid.shape)
df_obs_covid_qry = df_obs_covid.query('(DATE >= stdt_covid) and (DATE <= endt_covid or endt_covid == "") and DESCRIPTION.str.contains("Oxygen")')
print("Query:",df_obs_covid_qry.shape)
#print(df_obs_covid_qry)
#df_obs_covid_qry = df_obs_covid_qry.drop(['stdt_covid', 'endt_covid', 'REASONDESCRIPTION'], axis=1)
#df_obs_covid_qry.head()
#df_obs_grp = df_obs_covid_qry.groupby("PATIENT")["DESCRIPTION"].count()
#print(df_obs_grp.shape)

Abfrage in encounters_info nach zeitgleichem Auftreten mit COVID

In [ ]:
df_enc_new = pd.read_sql_query("SELECT PATIENT, START, STOP, ENCOUNTERCLASS, REASONDESCRIPTION FROM encounters_info", dwh_db)
enc_class = df_enc_new.groupby("ENCOUNTERCLASS")["PATIENT"].count()
# print(enc_class)

In [ ]:
df_enc_all = pd.read_sql_query("SELECT PATIENT, START, STOP, ENCOUNTERCLASS, REASONDESCRIPTION FROM encounters_info", dwh_db)
df_enc_covid = pd.merge(df_enc_all, df_conditions_filter, left_on="PATIENT", right_on="PATIENT",how="inner")
print("Merge:",df_enc_covid.shape)
df_enc_covid_qry = df_enc_covid.query('(STOP >= stdt_covid or STOP == "") and (START <= endt_covid or endt_covid == "") and (ENCOUNTERCLASS == "inpatient")')
print("Query:",df_enc_covid_qry.shape)
# print(df_enc_covid_qry)
# df_enc_covid_qry['Count_wellness'] = np.where(df_enc_covid_qry['ENCOUNTERCLASS'] == "wellnes", 0 , 1)
# print("Wellness = 1:",df_enc_covid_qry.shape)
# df_enc_covid_qry.head()
df_enc_grp = df_enc_covid_qry.groupby("PATIENT")["ENCOUNTERCLASS"].count().rename("inpatient")
# print(df_enc_grp)

# Factsheet erstellen
Als Grundlage wird Patients_info genommen. Das dataframe wird bereinigt und erweitert wie im DMP beschrieben.

In [ ]:
df_fs = pd.read_sql_query("SELECT * FROM patients_INFO WHERE DEATHDATE >= '2020-01-01' OR DEATHDATE == ''", dwh_db)
df_fs = df_fs.drop(['SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
        'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
        'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'], axis = 1)
df_county = pd.read_sql_query("SELECT COUNTY, DENSITY_per_sqm, SEASIDE FROM county_info", dwh_db)
df_county['SEASIDE'] = np.where(df_county['SEASIDE'] == 'Yes', 1,0)
df_covid = pd.read_sql_query("SELECT PATIENT, START AS stdt_covid, STOP AS endt_covid FROM conditions_info WHERE DESCRIPTION = 'COVID-19'", dwh_db)
# merge county and conditons = Covid into fact sheet with patients
df_fs = pd.merge(df_fs, df_county, left_on="COUNTY", right_on="COUNTY", how="inner")
df_fs = pd.merge(df_fs, df_covid, left_on="Id", right_on="PATIENT", how="inner")
df_fs = df_fs.drop(['PATIENT'], axis = 1)
# Calculate age when Covid infections occured
df_fs["BIRTHDATE"] = pd.to_datetime(df_fs["BIRTHDATE"])
df_fs["stdt_covid"] = pd.to_datetime(df_fs["stdt_covid"])
df_fs["AGE_Covid"] = df_fs.stdt_covid.dt.year - df_fs.BIRTHDATE.dt.year
# merge tables for severity score
df_fs = pd.merge(df_fs, df_dev_grp, left_on="Id", right_on="PATIENT", how="left")
df_fs = pd.merge(df_fs, df_med_grp, left_on="Id", right_on="PATIENT", how="left")
df_fs = pd.merge(df_fs, df_enc_grp, left_on="Id", right_on="PATIENT", how="left")
df_fs = pd.merge(df_fs, df_cond_grp, left_on="Id", right_on="PATIENT", how="left")
# Replace NaN with 0 in Score
df_fs["inpatient"] = df_fs.inpatient.fillna(0)
df_fs["Medications"] = df_fs.Medications.fillna(0)
df_fs["Ventilation"] = df_fs.Ventilation.fillna(0)
df_fs["Disorder"] = df_fs.Disorder.fillna(0)
df_fs["Death"] = np.where(df_fs['DEATHDATE'] == "", "Yes" , "No")
df_fs["SEVID"] = np.where(df_fs['inpatient'] == 0, 0, 1) + np.where(df_fs['Medications'] == 0, 0, 1) + np.where(df_fs['Ventilation'] == 0, 0, 1) + np.where(df_fs['Disorder'] == 0, 0, 1) + np.where(df_fs['DEATHDATE'] == "", 0 , 1)
# df_fs[df_fs['Death'] == 'Yes']
# print(df_fs.shape)
df_fs.head()
# df_fs.info()
# uncomment for general analysis
df_fs.describe()


# Statistik  
### chi-square Test
Default: Pearson’s chi-squared statistic   
pvalue < 0.05 zeigt einen Zusammenhang zwischen den Paramtern an  
### Alternative: Man whitney U Test  
Nicht geplant


In [ ]:
ct_den_sco = pd.crosstab(df_fs['DENSITY_per_sqm'], df_fs['SEVID'])
print(ct_den_sco)
chi2, p, dof, expected = stats.chi2_contingency(ct_den_sco)
print (f'Chi-square Statistic: {chi2}, p-value: {p}')

In [ ]:
ct_sea_sco = pd.crosstab(df_fs['SEASIDE'], df_fs['Ventilation'])
print(ct_sea_sco)
chi2, p, dof, expected = stats.chi2_contingency(ct_sea_sco)
print (f'Chi-square Statistic: {chi2}, p-value: {p}')

# Diagramme  
Heatmap für den Zusammenhang zwischen Sevid Score, Dichte und Wohnort an der Küste

In [ ]:
# Generating the correlating matrix
df_fs_heat = df_fs.drop(['LAT', 'LON', 'AGE_Covid', 'Medications',
       'inpatient', 'Disorder', 'Death'], axis = 1)
corr = df_fs_heat.corr()

plt.figure(figsize=(7, 5))
# Generating the correlation heat-map
sns.heatmap(corr, annot=False)

In [ ]:
#Generating Map

Map_Coordinates=(42.40, -71.38)
map_ma = folium.Map(location = Map_Coordinates, tiles="Cartodb Positron", width = "100%", zoom_start = 8)
coordinates = df_fs[["LAT", "LON"]].values.tolist()
for i in range (0, len(coordinates)):
  folium.RegularPolygonMarker(coordinates[i], radius = 0.5, fill_color = "blue").add_to(map_ma)
'''
geojson_data = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/us-states.json"
    #"https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/us_counties_20m_topo.json"
).json()

folium.GeoJson(geojson_data
               ).add_to(map_ma)

folium.LayerControl().add_to(map_ma)
'''
HeatMap(coordinates).add_to(map_ma)
map_ma

# Relevante Informationen zur Reproduzierbarkeit

## Versionen der Packages

In [ ]:
!python --version
print('Pandas: ', pd.__version__,'\n',
      'Numpy: ', np.__version__,'\n',
      'sqlite3: ', sqlite3.version,'\n',
      'seaborn: ', sns.__version__,'\n',
      'matplotlib: ', matplotlib.__version__,'\n',
      'Folium: ', folium.__version__)

## Hashwert des factsheets

In [ ]:
from pandas.util import hash_pandas_object
df_fs_hashes = hash_pandas_object(df_fs)
df_fs_hashes

Summe sollte lauten: 1167844697639228167

In [ ]:
df_fs_hashes.sum()